In [1]:
import csv
import pandas as pd
import numpy as np
from collections import Counter
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from numpy import array

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM

import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df


C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
len(d)

In [2]:
with open('frames.json') as f:
    d = json.load(f)

#lets put the data into a pandas df
#clicking on raw_nyc_phil.json under "Input Files"
#tells us parent node is 'programs'
text=[]
for i in range(len(d)):
    nycphil = json_normalize(d[i]['turns'])
    text.append(nycphil['text'])


In [3]:
from tqdm import tqdm
for i in tqdm(range(len(text))):
    l=len(text[i])
    if(l%2==1):
        del text[i][l-1]



100%|████████████████████████████████████| 1369/1369 [00:00<00:00, 3355.20it/s]


In [4]:
import itertools
merged = list(itertools.chain(*text))

In [5]:
df=pd.Series(merged)

In [6]:
message=[]
response=[]
for i in range(len(merged)):
    if(i%2==0):
        message.append(merged[i])
    else:
        response.append(merged[i])
       

In [7]:
text = pd.DataFrame(list(zip(message,response)), 
               columns =['message', 'response']) 

In [8]:
text.head(10)

,message,response
0,I'd like to book a trip to Atlantis from Capri...,"Hi...I checked a few options for you, and unfo..."
1,"Yes, how about going to Neverland from Caprica...",I checked the availability for this date and t...
2,I have no flexibility for dates... but I can l...,I checked the availability for that date and t...
3,"Hello, I am looking to book a vacation from Go...","Hi. Sorry, I can't find any trips from Gotham ..."
4,What about a trip from Gotham City to Neverlan...,"Sorry, I cannot find any trips leaving from Go..."
5,Would any packages to Mos Eisley be available ...,There are no trips available to Mos Eisley.
6,"You know what, I'd like to try and visit Never...",I cannot find any trips available to Neverland.
7,Do you have any trips from Gotham City to Kobe...,I can book you a 3 day trip to Kobe leaving fr...
8,"No, that's too far for me. I need a flight tha...",I can book you a trip from Birmingham to Kobe ...
9,How many days would I be in Kobe?,You would arrive in Kobe August 17th and retur...


In [ ]:
text.to_csv('text.csv',index=None)

In [ ]:
text=pd.read_csv('../input/data-travel/text.csv')
text.head()

In [9]:
stop_words = set(stopwords.words("english"))

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

from tqdm import tqdm
preprocessed_que = []
preprocessed_ans = []
# tqdm is for printing the status bar
for sentance in tqdm(text['message'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    preprocessed_que.append(sentance.strip())

for sentance in tqdm(text['response'].values):
    sentance = re.sub(r"http\S+", "", str(sentance))
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    preprocessed_ans.append(sentance.strip())

100%|████████████████████████████████████| 9579/9579 [00:02<00:00, 3383.42it/s]


In [10]:
text['message'] = preprocessed_que
text['response'] = preprocessed_ans
text.head()

,message,response
0,I would like to book a trip to Atlantis from C...,Hi I checked a few options for you and unfortu...
1,Yes how about going to Neverland from Caprica ...,I checked the availability for this date and t...
2,I have no flexibility for dates but I can leav...,I checked the availability for that date and t...
3,Hello I am looking to book a vacation from Got...,Hi Sorry I can not find any trips from Gotham ...
4,What about a trip from Gotham City to Neverlan...,Sorry I cannot find any trips leaving from Got...


In [11]:
train=text[0:8000]
test=text[8000: ]

In [12]:
train_input=train['message'].values
test_input=test['message'].values

train_target=train['response'].values
test_target=test['response'].values

In [13]:
max_input_len = 60

embed_size = 100

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(list(train_input) + list(test_input))

train_q = tokenizer.texts_to_sequences(train_input)
test_q = tokenizer.texts_to_sequences(test_input)

x_train = pad_sequences(train_q, maxlen=max_input_len)
x_test = pad_sequences(test_q, maxlen=max_input_len)


In [14]:
max_output_len=60
t = Tokenizer(num_words=500)
t.fit_on_texts(list(train_target)+list(test_target))

train_a = t.texts_to_sequences(train_target)
test_a = t.texts_to_sequences(test_target)

y_train = pad_sequences(train_a,maxlen=max_output_len)
y_test = pad_sequences(test_a,maxlen=max_output_len)

    
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [15]:
X1, X2, y = list(), list(), list()

target_in = y_train
    
src_encoded = to_categorical(x_train)
tar_encoded = to_categorical(y_train)
tar2_encoded = to_categorical(target_in)

X1.append(src_encoded)
X2.append(tar2_encoded)
y.append(tar_encoded)

X1=array(X1).reshape(8000,60,500)
X2=array(X2).reshape(8000,60,500)
y=array(y).reshape(8000,60,500)

print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

(8000, 60, 500) (8000, 60, 500) (8000, 60, 500)
X1=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 19, 29, 2, 26, 6, 42, 2, 12, 27, 449, 48, 8, 74, 1, 15, 6, 51, 18], X2=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 184, 7, 2, 267, 145, 8, 1, 5, 97, 32, 23, 49, 451, 9, 60, 102, 31, 20, 11, 1, 25, 4, 51, 72, 452, 93, 166], y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 184, 7, 2, 267, 145, 8, 1, 5, 97, 32, 23, 49, 451, 9, 60, 102, 31, 20, 11, 1, 25, 4, 51, 72, 452, 93, 166]


In [16]:
def define_models(n_input, n_output, n_units):
# define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
# define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
# define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
# return all models
    return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
    # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
    # store prediction
        output.append(yhat[0,0,:])
    # update state
        state = [h, c]
    # update target sequence
        target_seq = yhat
    return array(output)
 
# configure problem
n_features = 500
n_steps_in = 60
n_steps_out = 60
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 500)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 500)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 322048      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  322048      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [17]:
history=train.fit([X1, X2], y, epochs=200)

Epoch 1/200
5280/8000 [==================>...........] - ETA: 41:23 - loss: 6.1968 - acc: 0.0000e+ - ETA: 25:07 - loss: 6.1779 - acc: 0.0044   - ETA: 19:33 - loss: 6.1568 - acc: 0.27 - ETA: 16:54 - loss: 6.1366 - acc: 0.39 - ETA: 15:36 - loss: 6.1141 - acc: 0.47 - ETA: 14:26 - loss: 6.0906 - acc: 0.52 - ETA: 13:43 - loss: 6.0616 - acc: 0.57 - ETA: 13:06 - loss: 6.0289 - acc: 0.59 - ETA: 12:30 - loss: 5.9919 - acc: 0.61 - ETA: 11:57 - loss: 5.9405 - acc: 0.62 - ETA: 11:38 - loss: 5.8619 - acc: 0.64 - ETA: 11:21 - loss: 5.7565 - acc: 0.65 - ETA: 11:01 - loss: 5.6372 - acc: 0.66 - ETA: 10:43 - loss: 5.4940 - acc: 0.67 - ETA: 10:28 - loss: 5.3666 - acc: 0.68 - ETA: 10:23 - loss: 5.2265 - acc: 0.68 - ETA: 10:10 - loss: 5.0883 - acc: 0.69 - ETA: 9:57 - loss: 4.9643 - acc: 0.7015 - ETA: 10:36 - loss: 4.8197 - acc: 0.70 - ETA: 10:28 - loss: 4.6891 - acc: 0.71 - ETA: 10:18 - loss: 4.5758 - acc: 0.71 - ETA: 13:02 - loss: 4.4574 - acc: 0.72 - ETA: 12:43 - loss: 4.3364 - acc: 0.72 - ETA: 12:28 - l

KeyboardInterrupt: 

In [ ]:
X_1, X_2, Y = list(), list(), list()

target_in = y_test
    
src_encoded = to_categorical(x_test,num_classes=500)
tar_encoded = to_categorical(y_test,num_classes=500)
tar2_encoded = to_categorical(target_in,num_classes=500)

X_1.append(src_encoded)
X_2.append(tar2_encoded)
Y.append(tar_encoded)

X_1=array(X_1).reshape(1579,60,500)
X_2=array(X_2).reshape(1579,60,500)
Y=array(Y).reshape(1579,60,500)


In [ ]:
x1=X_1[0:1:,]
x2=X_1[1:2:,]
x3=X_1[2:3:,]
x4=X_1[3:4:,]

In [ ]:
target = predict_sequence(infenc, infdec, x4, n_steps_out, n_features)
print('X=%s y=%s, yhat=%s' % (one_hot_decode(X_1[3]), one_hot_decode(Y[3]),one_hot_decode(target)))
y_pred=one_hot_decode(target)
a=t.word_index
b=list(a.keys())
reply=[]
for i in range(6):
    for j in range(6):
        if y_pred[j]==i+1:
            reply.append(b[i])

print("*****************EMAIL_BODY********************\n")

print(test_input[3])

print("*******************Suggested Reply**************")

print(' '.join(reply))

print("*******************Actual Reply******************")
print(test_target[3])